In [64]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import HeatMap
from geopy.distance import geodesic
from folium import Element


# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'  # 윈도우: 맑은 고딕
# plt.rcParams['font.family'] = 'Nanum Gothic'  # Mac/Linux: 나눔 고딕

# 폰트 크기, 스타일 등 기본값 설정 (선택 사항)
plt.rcParams['axes.unicode_minus'] = False  # 음수(-) 부호가 깨지지 않도록 설정

ilsan_mart_file = "C:/Users/saeya/Downloads/dataset_life_infra_일산_대규모점포_인허가일자포함.csv"
ilsan_restaurant_file = 'C:/Users/saeya/Downloads/dataset_life_infra_일산_일반음식점_인허가일자포함.csv'
ilsan_hospital_file = "C:/Users/saeya/Downloads/dataset_life_infra_일산_병원.csv"
ilsan_hospital2_file="C:/Users/saeya/Downloads/dataset_life_infra_일산_의원.csv"



bundang_mart_file = "C:/Users/saeya/Downloads/dataset_life_infra_분당_대규모점포_인허가일자포함 (1).csv"
bundang_restaurant_file ="C:/Users/saeya/Downloads/dataset_life_infra_분당_일반음식점_인허가일자포함.csv"
bundang_hospital_file = "C:/Users/saeya/Downloads/dataset_life_infra_분당_병원 (1).csv"
bundang_hospital2_file="C:/Users/saeya/Downloads/dataset_life_infra_분당_의원 (1).csv"


ilsan_mart_data = pd.read_csv(ilsan_mart_file)
ilsan_restaurant_data = pd.read_csv(ilsan_restaurant_file)
ilsan_hospital_data = pd.read_csv(ilsan_hospital_file)
ilsan_hospital2_data = pd.read_csv(ilsan_hospital2_file)

bundang_mart_data = pd.read_csv(bundang_mart_file)
bundang_restaurant_data = pd.read_csv(bundang_restaurant_file)
bundang_hospital_data = pd.read_csv(bundang_hospital_file)
bundang_hospital2_data= pd.read_csv(bundang_hospital2_file)

ilsan_mart_data['폐업일자'] = pd.to_datetime(ilsan_mart_data['폐업일자'].astype(str).str.replace('-', ''), errors='coerce')


In [26]:
def count_without_closing_date(file_path):
    # CSV 파일 불러오기
    df = pd.read_csv(file_path)
    
    # '폐업일자'가 없는 경우 처리
    if '폐업일자' in df.columns:
        df = df[df['폐업일자'].isna()]  # 폐업일자가 없는 데이터만 필터링

    # '영업상태명' 조건 추가
    if '영업상태명' in df.columns:
        df = df[(df['영업상태명'] == '정상영업') | (df['영업상태명'] == '영업개시전') | (df['영업상태명'] == '영업')]
    
    # '상세영업상태명' 조건 추가
    if '상세영업상태명' in df.columns:
        df = df[(df['상세영업상태명'] == '정상영업') | (df['상세영업상태명'] == '영업개시전') | (df['상세영업상태명'] == '영업중') | (df['상세영업상태명'] == '영업')]

    # 필터링된 데이터 반환
    return df

In [28]:
filtered_ilsan_mart_data= count_without_closing_date(ilsan_mart_file)
filtered_ilsan_restaurant_data = count_without_closing_date(ilsan_restaurant_file)
filtered_ilsan_hospital_data = count_without_closing_date(ilsan_hospital_file)
filtered_ilsan_hospital2_data = count_without_closing_date(ilsan_hospital2_file)

filtered_bundang_mart_data = count_without_closing_date(bundang_mart_file)
filtered_bundang_restaurant_data = count_without_closing_date(bundang_restaurant_file)
filtered_bundang_hospital_data = count_without_closing_date(bundang_hospital_file)
filtered_bundang_hospital2_data = count_without_closing_date(bundang_hospital2_file)
# 필터링된 데이터 개수 출력
print("필터링된 일산 마트 데이터 개수:", filtered_ilsan_mart_data.shape[0])
print("필터링된 일산 음식점 데이터 개수:", filtered_ilsan_restaurant_data.shape[0])
print("필터링된 일산 병원 데이터 개수:", filtered_ilsan_hospital_data.shape[0])
print("필터링된 일산 병원2 데이터 개수:", filtered_ilsan_hospital2_data.shape[0])

print("필터링된 분당 마트 데이터 개수:", filtered_bundang_mart_data.shape[0])
print("필터링된 분당 음식점 데이터 개수:", filtered_bundang_restaurant_data.shape[0])
print("필터링된 분당 병원 데이터 개수:", filtered_bundang_hospital_data.shape[0])
print("필터링된 분당 병원2 데이터 개수:", filtered_bundang_hospital2_data.shape[0])

필터링된 일산 마트 데이터 개수: 41
필터링된 일산 음식점 데이터 개수: 5721
필터링된 일산 병원 데이터 개수: 49
필터링된 일산 병원2 데이터 개수: 737
필터링된 분당 마트 데이터 개수: 46
필터링된 분당 음식점 데이터 개수: 4639
필터링된 분당 병원 데이터 개수: 23
필터링된 분당 병원2 데이터 개수: 830


In [105]:
# 일산 마트 데이터에서 위도와 경도 추출
ilsan_mart_latitudes = filtered_ilsan_mart_data['WGS84위도']
ilsan_mart_longitudes = filtered_ilsan_mart_data['WGS84경도']

# 일산 음식점 데이터에서 위도와 경도 추출
ilsan_restaurant_latitudes = filtered_ilsan_restaurant_data['위도']
ilsan_restaurant_longitudes = filtered_ilsan_restaurant_data['경도']

# 일산 병원 데이터에서 위도와 경도 추출
ilsan_hospital_latitudes = filtered_ilsan_hospital_data['위도']
ilsan_hospital_longitudes = filtered_ilsan_hospital_data['경도']

# 일산 병원2 데이터에서 위도와 경도 추출
ilsan_hospital2_latitudes = filtered_ilsan_hospital2_data['위도']
ilsan_hospital2_longitudes = filtered_ilsan_hospital2_data['경도']

# 분당 마트 데이터에서 위도와 경도 추출
bundang_mart_latitudes = filtered_bundang_mart_data['WGS84위도']
bundang_mart_longitudes = filtered_bundang_mart_data['WGS84경도']

# 분당 음식점 데이터에서 위도와 경도 추출
bundang_restaurant_latitudes = filtered_bundang_restaurant_data['위도']
bundang_restaurant_longitudes = filtered_bundang_restaurant_data['경도']

# 분당 병원 데이터에서 위도와 경도 추출
bundang_hospital_latitudes = filtered_bundang_hospital_data['위도']
bundang_hospital_longitudes = filtered_bundang_hospital_data['경도']

# 분당 병원2 데이터에서 위도와 경도 추출
bundang_hospital2_latitudes = filtered_bundang_hospital2_data['위도']
bundang_hospital2_longitudes = filtered_bundang_hospital2_data['경도']


In [107]:
# 일산 마트 데이터로 DataFrame 생성
ilsan_mart = pd.DataFrame({'위도': ilsan_mart_latitudes, 
                           '경도': ilsan_mart_longitudes, 
                           '타입': '일산_대규모점포'})

# 일산 음식점 데이터로 DataFrame 생성
ilsan_restaurant = pd.DataFrame({'위도': ilsan_restaurant_latitudes, 
                                 '경도': ilsan_restaurant_longitudes, 
                                 '타입': '일산_음식점'})

# 일산 병원 데이터로 DataFrame 생성
ilsan_hospital = pd.DataFrame({'위도': ilsan_hospital_latitudes, 
                               '경도': ilsan_hospital_longitudes, 
                               '타입': '일산_병원'})

# 일산 병원2 데이터로 DataFrame 생성
ilsan_hospital2 = pd.DataFrame({'위도': ilsan_hospital2_latitudes, 
                                '경도': ilsan_hospital2_longitudes, 
                                '타입': '일산_병원2'})

# 분당 마트 데이터로 DataFrame 생성
bundang_mart = pd.DataFrame({'위도': bundang_mart_latitudes, 
                             '경도': bundang_mart_longitudes, 
                             '타입': '분당_대규모점포'})

# 분당 음식점 데이터로 DataFrame 생성
bundang_restaurant = pd.DataFrame({'위도': bundang_restaurant_latitudes, 
                                   '경도': bundang_restaurant_longitudes, 
                                   '타입': '분당_음식점'})

# 분당 병원 데이터로 DataFrame 생성
bundang_hospital = pd.DataFrame({'위도': bundang_hospital_latitudes, 
                                 '경도': bundang_hospital_longitudes, 
                                 '타입': '분당_병원'})

# 분당 병원2 데이터로 DataFrame 생성
bundang_hospital2 = pd.DataFrame({'위도': bundang_hospital2_latitudes, 
                                  '경도': bundang_hospital2_longitudes, 
                                  '타입': '분당_병원2'})

print(ilsan_mart)

           위도          경도        타입
0   37.659901  126.754939  일산_대규모점포
1   37.666098  126.750520  일산_대규모점포
2   37.665939  126.756403  일산_대규모점포
3   37.642359  126.792682  일산_대규모점포
4   37.667964  126.751387  일산_대규모점포
5   37.667321  126.767019  일산_대규모점포
6   37.643193  126.789720  일산_대규모점포
7   37.673157  126.755212  일산_대규모점포
8   37.667979  126.751624  일산_대규모점포
9   37.651875  126.778808  일산_대규모점포
10  37.667740  126.764546  일산_대규모점포
11  37.685654  126.770872  일산_대규모점포
12  37.659065  126.768867  일산_대규모점포
13  37.647881  126.782400  일산_대규모점포
14  37.648830  126.779292  일산_대규모점포
15  37.668543  126.765405  일산_대규모점포
16  37.695229  126.763578  일산_대규모점포
17  37.681612  126.778512  일산_대규모점포
18  37.655798  126.771928  일산_대규모점포
19  37.677498  126.769095  일산_대규모점포
20  37.640774  126.788516  일산_대규모점포
21  37.648830  126.779292  일산_대규모점포
22  37.664376  126.754436  일산_대규모점포
23  37.670904  126.761102  일산_대규모점포
24  37.678767  126.764409  일산_대규모점포
25  37.687742  126.772873  일산_대규모점포
26  37.661026  126.768319  일

In [114]:
db = DBSCAN(eps=0.002, min_samples=3)  # eps는 밀도 기준 거리, min_samples는 클러스터 최소 크기

ilsan_mart['클러스터'] = db.fit_predict(ilsan_mart[['위도', '경도']])# 일산 마트 데이터에서 클러스터링

# 일산 음식점 데이터에서 클러스터링
ilsan_restaurant['클러스터'] = db.fit_predict(ilsan_restaurant[['위도', '경도']])

# 일산 병원 데이터에서 클러스터링
ilsan_hospital['클러스터'] = db.fit_predict(ilsan_hospital[['위도', '경도']])

# 일산 병원2 데이터에서 클러스터링
ilsan_hospital2['클러스터'] = db.fit_predict(ilsan_hospital2[['위도', '경도']])


In [159]:
m = folium.Map(location=[37.6584, 126.7686],tiles='cartodbpositron', zoom_start=13)
legend_html = """
<div style="
    position: fixed;
    bottom: 50px;
    left: 50px;
    width: 230px;
    height: 60px;
    background-color: white;
    border:2px solid grey;
    z-index:9999;
    font-size:14px;
    padding: 10px;">
    <strong>일산신도시</strong><br>
    <i style="background:blue; width:10px; height:10px; float:left; margin-right:5px; display:inline-block;"></i> 일산신도시 대규모점포 <br>
</div>
"""

legend = Element(legend_html)
m.get_root().html.add_child(legend)


# 히트맵 데이터 준비
heat_data = [[row['위도'], row['경도']] for idx, row in ilsan_mart.iterrows()]

# 히트맵 추가
HeatMap(heat_data).add_to(m)
m.save('C:/Users/saeya/Downloads/ilsan_mart_heatmap.html')
m 